如何更好的访问并控制模型内部过程?

如何选择最佳轮次?
- 训练足够多的轮次, 根据模型开始过拟合的轮次确定训练模型需要的正确轮次, 然后使用这个最佳轮次, 从头再启动一次训练
- 当观测到验证损失不再降低时停止训练. 通过keras回调函数实现.
回调函数是在调用fit时传入模型的对象, 在训练过程中的不同时间点都会被模型调用. 可以访问关于模型状态与性能的所有可用数据, 还可以在训练过程中采取行动: 中断训练, 保存模型, 加载一组不同的权重或改变模型的状态. 它的一些用法示例如下所是:  
- 模型检查点: 在训练过程中的不同时间点保存模型的当前权重
- 提前终止: 如果验证损失不再改善, 则中断训练, 同时保存训练过程中的最佳模型
- 训练过程中动态调节某些参数: 比如优化器的学习率
- 训练过程中记录训练指标和验证指标, 或将模型学习到的表示可视化(表示也在不断更新)
keras.callbacks模块包含许多内置的回调函数, 有:
```python
import keras.callbacks as kc
kc.ModelCheckpoint
kc.EarlyStopping
kc.LearningRateScheduler
kc.ReduceLROnPlateau
kc.CSVLogger
```

In [ ]:
# ModelCheckpoint与EarlyStopping回调函数
import keras

callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor='acc',              # 监控模型的验证精度
        patience=1,                 # 如果精度在多于一轮(即两轮)内不再改善,中断训练
        ),
    keras.callbacks.ModelCheckpoint( # 每轮过后保存当前权重
        filepath='my_model.h5',      # 目标模型文件的保存路径
        monitor='val_loss',
        save_best_only=True,         # 如果val_loss没有改善, 则不需要覆盖模型文件
        )
    ]

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])       # 如果回调函数中监控acc, 则acc应作为模型指标的一部分
model.fit(x, y,
          epochs=10,
          batch_size=32,
          callbacks=callbacks_list,
          validation_data=(x_val, y_val)) # 由于回调函数要监控val_loss和val_acc
                                          # 所以在fit时, 要传入validation_data

In [ ]:
# ReduceLROnPlateau回调函数
# 如果val_loss不再改善, 那么可以使用ReduceLRonPlateau降低学习率
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',            # 监控模型的验证损失
        factor=0.1,                    # 触发时将学习率除以10
        patience=10)                   # 如果验证损失10轮内都没有改善, 触发这个函数
    ]

model.fit(x, y,
          epochs=10,
          batch_size=32,
          callbacks=callbacks_list,         # 因为回调函数需要监控val_loss
          validation_data=(x_val, y_val))   # fit时需要传入validation_data

定义自己的回调函数, 创建keras.callbacks.Callback类的字类, 然后可以实现下述方法:
- on_epoch_begin
- on_epoch_end
- on_batch_begin
- on_batch_end
- on_train_begin
- on_train_end
上述方法被调用时都有一个logs参数, 该参数包含前一个批量, 前一个轮次或前一次训练的信息, 即训练指标和验证指标等. 此外, 回调函数还可以访问下列属性:
- self.model: 调用回调函数的模型实例
- self.validation_data: 传入fit函数作为验证数据的值

In [ ]:
# 自定义回调函数
import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):

    def set_model(self, model):
        # 训练之前由父模型调用, 告诉回调函数是哪个模型在调用它
        self.model = model
        layer_output = [layer.output for layer in model.layers]
        # 返回模型每层的激活
        self.activations_model = keras.models.Model(model.input,
                                                    layer_output)

    def on_epoch_end(self, epochs, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Require validation_data')
        validation_sample = self.validation_data[0][0:1]
        activations = self.activations_model.predict(validation_sample)
        with open('activations_at_epoch_' + str(epoch) + '.npz', 'w') as f:
            np.savez(f, activations)

TensorBoard:
内置于tensorflow中的基于浏览器的可视化工具(只有keras使用tensorflow后端时, 这一方法才能用于keras模型.
主要用途: 可视化训练过程中模型内部发生的一切
- 可视化训练过程中的各项监控指标
- 可视化模型架构
- 将激活和梯度的直方图可视化
- 以三维的形式研究嵌入

In [ ]:
# 可视化imdb的文本分类模型
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 2000
max_len = 500
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128,
                           input_length=max_len, name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='my_log_dir',        # 日志文件保存位置
        histogram_freq=1,            # 每一轮之后记录激活直方图
        embeddings_freq=1,           # 每一轮之后记录嵌入数据
        )
    ]

history = model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=callbacks)

在命令行启动TensorBoard服务器, 指示它读取回调函数当前正在写入的日至
$ tensorboard --logdir=my_log_dir
然后在浏览器打开 http://localhost:6006, 并查看模型的训练过程.

keras 还提供了另一种方式 keras.utils.plot_model 将模型绘制为层组成的图, 而不是TensorFlow运算组成的图.
from keras.utils import plot_model
plot_model(model, to_file='model.png')

- keras回调函数提供了一种简单的方法, 可以在训练过程中监控模型,并根据模型状态自动采取行动(调整学习率, 提前终止)
- TensorBoard 可以在浏览器中监控模型, 并将模型活动可视化.